In [1]:
import re
import pandas as pd
import numpy as np
import seaborn as sns

## Data cleaning 

In [2]:
# Load data
df = pd.read_csv('datasets/scraped.csv')
df

,code,number,price,city
0,3,344,"229,000 AED",uae_sharjah
1,U,1313,"109,000 AED",duabi_new
2,H,3222,"72,000 AED",duabi_new
3,3,4441,"38,000 AED",uae_sharjah
4,3,7776,"38,000 AED",uae_sharjah
...,...,...,...,...
2741,X,81686,"AED2,000",duabi_new
2742,X,91393,"AED2,000",duabi_new
2743,X,91494,"AED2,000",duabi_new
2744,X,91595,"AED2,000",duabi_new


In [3]:
# Remove AED and commans from price
df['price'] = df['price'].replace(r"[a-zA-Z,]",'',regex = True).astype(int)
df

,code,number,price,city
0,3,344,229000,uae_sharjah
1,U,1313,109000,duabi_new
2,H,3222,72000,duabi_new
3,3,4441,38000,uae_sharjah
4,3,7776,38000,uae_sharjah
...,...,...,...,...
2741,X,81686,2000,duabi_new
2742,X,91393,2000,duabi_new
2743,X,91494,2000,duabi_new
2744,X,91595,2000,duabi_new


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2746 entries, 0 to 2745
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    2746 non-null   object
 1   number  2746 non-null   int64 
 2   price   2746 non-null   int32 
 3   city    2746 non-null   object
dtypes: int32(1), int64(1), object(2)
memory usage: 75.2+ KB


In [5]:
# Check for repeating digits
df['number'].nunique()

# Seems to me there are repeating numbers.
# Will only keep the latest assuming its latest price. 

2387

In [6]:
# Check if they have different codes
df[df.duplicated('number', keep = False)]

# Looks like same number but with different code so no need to drop

,code,number,price,city
24,3,44400,29000,uae_sharjah
41,H,40333,10900,duabi_new
43,L,40333,10900,duabi_new
44,N,40333,10900,duabi_new
45,P,40333,10900,duabi_new
...,...,...,...,...
1906,U,88373,2400,duabi_new
1909,U,88747,2400,duabi_new
1912,U,92282,2400,duabi_new
1922,U,99373,2400,duabi_new


In [7]:
# Check number of duplicated rows, >1 == duplicate
df.groupby(df.columns.tolist(),as_index=False).size()['size'].nunique()

# No need to drop any rows

1

In [8]:
# Fix the city columns
df['city'].unique()

array(['uae_sharjah', 'duabi_new', 'ajman_plate', 'oaq_plate'],
      dtype=object)

In [9]:
# Replace dict
city_replace = {
    'uae_sharjah':'Sharjah',
    'duabi_new':'Dubai',
    'ajman_plate':'Ajman',
    'oaq_plate':'OAQ'
}

# replace
df['city'] = df['city'].replace(city_replace)
df

,code,number,price,city
0,3,344,229000,Sharjah
1,U,1313,109000,Dubai
2,H,3222,72000,Dubai
3,3,4441,38000,Sharjah
4,3,7776,38000,Sharjah
...,...,...,...,...
2741,X,81686,2000,Dubai
2742,X,91393,2000,Dubai
2743,X,91494,2000,Dubai
2744,X,91595,2000,Dubai


In [10]:
# Check value counts
df['city'].value_counts()

# There isn't enough data to train on non 
# Dubai number plates, so for EDA and training, I'll only use Dubai plates

df = df[df['city'] == 'Dubai']
df = df[['number','code','price']]
df

,number,code,price
1,1313,U,109000
2,3222,H,72000
27,30033,P,24000
28,7765,L,17500
29,8876,U,17500
...,...,...,...
2741,81686,X,2000
2742,91393,X,2000
2743,91494,X,2000
2744,91595,X,2000


## Check old dataset and maybe merge

In [11]:
# Need to cross check older values
df1 = pd.read_csv('datasets/num_plates.csv')
df1.head()

,Plates,Number,Code,Price
0,"274 I 215,000AED",274,I,"215,000AED"
1,"78888 H 72,000AED",78888,H,"72,000AED"
2,"6777 K 72,000AED",6777,K,"72,000AED"
3,"3334 R 72,000AED",3334,R,"72,000AED"
4,"22232 N 41,000AED",22232,N,"41,000AED"


In [12]:
# Beat it into shape
df1 = df1.rename(columns = {col:col.lower() for col in df1.columns})
df1.head()

,plates,number,code,price
0,"274 I 215,000AED",274,I,"215,000AED"
1,"78888 H 72,000AED",78888,H,"72,000AED"
2,"6777 K 72,000AED",6777,K,"72,000AED"
3,"3334 R 72,000AED",3334,R,"72,000AED"
4,"22232 N 41,000AED",22232,N,"41,000AED"


In [13]:
# Even more
df1['price'] = df1['price'].replace(r"[a-zA-Z,]",'',regex = True).astype(int)
df1 = df1[['number','code','price']]
df1.head()

,number,code,price
0,274,I,215000
1,78888,H,72000
2,6777,K,72000
3,3334,R,72000
4,22232,N,41000


In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457 entries, 0 to 1456
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   number  1457 non-null   int64 
 1   code    1457 non-null   object
 2   price   1457 non-null   int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 28.6+ KB


In [15]:
# Append it to previous dataframe to extend # numbers
dfm = pd.concat([df,df1])
dfm = dfm.reset_index(drop=True)
dfm

,number,code,price
0,1313,U,109000
1,3222,H,72000
2,30033,P,24000
3,7765,L,17500
4,8876,U,17500
...,...,...,...
4146,98828,U,2400
4147,99232,U,2400
4148,99373,U,2400
4149,99585,U,2400


In [16]:
# Get rid of repeating plates, and keep the first instance,
# This is because I appended the old dataset to the new one
# So the previous value that appears is a more accurate price
# Seems to me we were able to add ~200 number plates by mergin the datasets
dfd = dfm.drop_duplicates(subset=['number', 'code'], keep='first').reset_index(drop=True)
dfd

,number,code,price
0,1313,U,109000
1,3222,H,72000
2,30033,P,24000
3,7765,L,17500
4,8876,U,17500
...,...,...,...
2963,26277,U,2400
2964,28299,U,2400
2965,63383,U,2400
2966,93966,U,2400


In [31]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2968 entries, 0 to 2967
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   number  2968 non-null   int64 
 1   code    2968 non-null   object
 2   price   2968 non-null   int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 58.1+ KB


In [17]:
# Save it for EDA 
# dfd.to_csv('datasets/cleaned.csv', index = False)

### Load dubizzle data

In [21]:
dub_arr = np.load('datasets/dub_scraped.npy')
dub_arr

array([['11000', 'K', 'AED 255,000'],
       ['1114', 'D', 'AED 105,000'],
       ['66000', 'K', 'AED 160,000'],
       ...,
       ['91181', 'R', 'AED 6,500'],
       ['8704', 'Q', 'AED 6,600'],
       ['93603', 'A', 'AED 2,900']], dtype='<U13')

In [29]:
# Pull out the values into arrays
dnum_arr = dub_arr[:,0]
dcode_arr = dub_arr[:,1]
dprice_arr = dub_arr[:,2]

df_dub = pd.DataFrame({'number':dnum_arr,
                       'code':dcode_arr,
                       'price':dprice_arr})
df_dub

,number,code,price
0,11000,K,"AED 255,000"
1,1114,D,"AED 105,000"
2,66000,K,"AED 160,000"
3,60007,Q,"AED 31,000"
4,8773,M,"AED 15,000"
...,...,...,...
975,7210,Q,"AED 9,200"
976,6353,E,"AED 15,000"
977,91181,R,"AED 6,500"
978,8704,Q,"AED 6,600"


In [30]:
df_dub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   number  980 non-null    object
 1   code    980 non-null    object
 2   price   980 non-null    object
dtypes: object(3)
memory usage: 23.1+ KB


In [32]:
# Get rid of duplicates
df_dub = df_dub.drop_duplicates(subset=['number', 'code'], keep='first').reset_index(drop=True)
df_dub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   number  885 non-null    object
 1   code    885 non-null    object
 2   price   885 non-null    object
dtypes: object(3)
memory usage: 20.9+ KB


In [33]:
# Fix datatypes
df_dub['number'] = df_dub['number'].astype(int)
df_dub['price'] = df_dub['price'].replace(r"[a-zA-Z,]",'',regex = True).astype(int)

df_dub

,number,code,price
0,11000,K,255000
1,1114,D,105000
2,66000,K,160000
3,60007,Q,31000
4,8773,M,15000
...,...,...,...
880,7210,Q,9200
881,6353,E,15000
882,91181,R,6500
883,8704,Q,6600


In [34]:
dfd

,number,code,price
0,1313,U,109000
1,3222,H,72000
2,30033,P,24000
3,7765,L,17500
4,8876,U,17500
...,...,...,...
2963,26277,U,2400
2964,28299,U,2400
2965,63383,U,2400
2966,93966,U,2400


In [35]:
# Final dataframe
dff = pd.concat([dfd,df_dub])
dff = dff.reset_index(drop=True)
dff

,number,code,price
0,1313,U,109000
1,3222,H,72000
2,30033,P,24000
3,7765,L,17500
4,8876,U,17500
...,...,...,...
3848,7210,Q,9200
3849,6353,E,15000
3850,91181,R,6500
3851,8704,Q,6600


In [36]:
dff = dff.drop_duplicates(subset=['number', 'code'], keep='first').reset_index(drop=True)
dff

,number,code,price
0,1313,U,109000
1,3222,H,72000
2,30033,P,24000
3,7765,L,17500
4,8876,U,17500
...,...,...,...
3835,7210,Q,9200
3836,6353,E,15000
3837,91181,R,6500
3838,8704,Q,6600


In [37]:
# Save it for EDA 
dff.to_csv('datasets/cleaned.csv', index = False)